In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
df = pd.read_csv('data/draftstars.csv')
df = df.groupby('Name').head(1).reset_index(drop=True) # How to add this constraint ???
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [3]:
def player_count_constraint(solver):
    ct = solver.Constraint(9, 9, 'total_players')
    
    for x in player_ix:
        ct.SetCoefficient(player_ix[x], 1)
        
    return solver

In [4]:
def position_constraint(solver):

    df['idx_ls'] = [[el] for el in df.index]
    pos_dict = df.groupby('position')['idx_ls'].agg('sum')
    pos_dict = dict(zip(list(pos_dict.index), list(pos_dict.values)))
    
    for pos in pos_dict:
        pos_count = 1 if pos == 'C' else 2
        ct = solver.Constraint(pos_count, pos_count, 'player_pos_' + pos)
        
        for ix in pos_dict[pos]:
            ct.SetCoefficient(player_ix[ix], 1)
            
    return solver

In [5]:
def salary_constraint(solver):
    ct = solver.Constraint(0, 100000, 'total_salary')
    
    for ix in player_ix:
        ct.SetCoefficient(player_ix[ix], int(df.iloc[ix]['salary']))
        
    return solver

In [6]:
def add_constraints(solver):
    solver = player_count_constraint(solver)
    solver = position_constraint(solver)
    solver = salary_constraint(solver)
    return solver

In [7]:
def set_obj_function(solver, player_ix):
    objective = solver.Objective()
    
    for ix in player_ix:
        objective.SetCoefficient(player_ix[ix], df.iloc[ix]['form'])
        
    objective.SetMaximization()
    solver.Solve()
    
    return solver, player_ix, objective

In [8]:
solver = add_constraints(solver)
solver, player_ix, objective = set_obj_function(solver, player_ix)
df_team = df.iloc[[bool(int(el.solution_value())) for el in player_ix.values()]]

print(f'''
    Generated team total PER: {round(df_team['form'].sum(), 2)}. \n 
    Total Team's salary:  ${round(df_team['salary'].sum(), 2)} 
''')

df_team


    Generated team total PER: 320.6. 
 
    Total Team's salary:  $98840 



,player_id,position,name,team,opponent,salary,fppg,form,playing_status,idx_ls
8,7000907,PF,Jerami Grant,Portland Trail Blazers,Oklahoma City Thunder,12950,32.86,40.92,EXPECTED STARTER,[8]
34,7001426,PG,Malcolm Brogdon,Portland Trail Blazers,Oklahoma City Thunder,12240,30.27,44.92,EXPECTED STARTER,[34]
35,7001429,SG,Buddy Hield,Indiana Pacers,Denver Nuggets,11590,24.69,36.42,EXPECTED STARTER,[35]
38,7001442,SF,Brandon Ingram,New Orleans Pelicans,Utah Jazz,14750,38.19,45.83,EXPECTED STARTER,[38]
53,7022332,PG,Jalen Brunson,New York Knicks,Brooklyn Nets,18680,43.93,57.67,EXPECTED STARTER,[53]
81,7022680,SG,Isaiah Joe,Oklahoma City Thunder,Portland Trail Blazers,6930,16.51,21.92,NaN,[81]
138,7023023,C,Walker Kessler,Utah Jazz,New Orleans Pelicans,9700,25.53,27.83,NaN,[138]
147,7023105,PF,Jarace Walker,Indiana Pacers,Denver Nuggets,6000,11.18,25.92,NaN,[147]
157,7023130,SF,Jalen Wilson,Brooklyn Nets,New York Knicks,6000,9.23,19.17,NaN,[157]
